In [56]:
import os
import sys
import findspark
findspark.init()
import random as rn
import numpy as np
import math
import sys
from functools import reduce
from pyspark import SparkContext,SparkConf,AccumulatorParam
conf=SparkConf()
sc = SparkContext(conf=conf ,master="local[*]")
#LSH-(r,cr,k,m) Family 
n=2**10
d=2**7
r=d/2
c=3/2
k=int(math.log(n,2)/2)
m=50
band=50

class ListAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue):
        return initialValue
    def addInPlace(self, v1, v2):
        v1.append(v2)
        return v1

data=[[rn.randint(0,1) for i in range(d)] for j in range(n)]

query=[rn.randint(0,1) for i in range(d)]

def bin_to_dec(Bin):
    s=0
    for i in range(len(Bin)):
        if Bin[i]!=0:
            s+=2**i
    return s
    
def reduce_(x,y):
    s=[]
    for i in zip(x,y):
        s.append(i[0]+i[1])
    return s

H_Family=[i for i in range(k)]

def H(x,i):
    return x[i]

def create_Signatures(x,H_Family):
    Hashed_Data=[]
    H_Family=rn.sample(H_Family,k)
    Hashed_Row=[]
    for item in x:
        #H_Family=rn.sample(H_Family,k)
        Hashed_Row.append([H(item,j) for j in H_Family])
    return Hashed_Row
try:

    signatures=sc.parallelize([data for i in range(m)]).map(lambda x:create_Signatures(x,H_Family))

    def partitioning_To_Bands(Sign_Data):#partition the Rdd
        if band==m:
            return Sign_Data
        else:
            bands=[]
            size=m//band
            for i in range(0,len(Sign_Data),size):
                reduced_Data=reduce(lambda x,y:reduce_(x,y),Sign_Data[i:i+size])
                bands.append(reduced_Data)
            return bands

    Tables=sc.accumulator(list(), ListAccumulatorParam())
    #Sign_Data=partitioning_To_Bands(signatures.collect())

    def Hash_To_Buckets(row):
        Bucket={}
        for i in range(len(row)):
            key=bin_to_dec(row[i])
            if key not in Bucket.keys():
                Bucket[key]=[data[i]]
            elif data[i] not in Bucket[key]:
                Bucket[key]+=[data[i]]
        Tables.add(Bucket)
    signatures.foreach(Hash_To_Buckets)
    #sc.parallelize(Sign_Data).foreach(Hash_To_Buckets)  
    def Nearest_Neighbors(bucket,query,r):
        NN=[]
        global H_Family
        H_Family=rn.sample(H_Family,k)
        query_key=bin_to_dec([H(query,j) for j in H_Family])
        if query_key in bucket.keys():
            NN=[]
            neighs=bucket[query_key]
            for neigh in neighs:
                dist=sum(np.abs(np.array(query)-np.array(neigh)))
                if dist<r:
                    NN.append(neigh)
        return NN
    results=sc.parallelize(Tables.value).map(lambda bucket:Nearest_Neighbors(bucket,query,r))
    results_LSH=list(map(lambda x:len(x),results.collect()))

except:
    
    sc.stop()
    
def linear_search():
    NN=[]
    FP=[]
    ANN=[]
    for item in data:
        dist=sum(np.abs(np.array(query)-np.array(item)))
        if dist<r:
                NN.append(item)
        elif r<=dist<=c*r:
                ANN.append(item)
        elif dist>=c*r:
                FP.append(item)
    return [NN,ANN,FP]

result_linear_search=linear_search()

print('Linear search results:\nNumber of NN with dist < r: ',len(result_linear_search[0]))
print('LSH results:\nNumber of NN with dist < r: ',sum(results_LSH))

sc.stop()

Linear search results:
Number of NN with dist < r:  453
LSH results:
Number of NN with dist < r:  354
